<a href="https://colab.research.google.com/github/AzlinRusnan/Sleep_Quality_Analysis/blob/main/Sleep_Duration_vs_Quality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Data Description**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [61]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

file_path = '/content/gdrive/MyDrive/Sleep_health_and_lifestyle_dataset.csv'
df = pd.read_csv(file_path)

In [62]:
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


##### **Columns Explanation:**

1. Person ID: An identifier for each individual.
2. Gender: The gender of the person (Male/Female).
3. Age: The age of the person in years.
4. Occupation: The occupation or profession of the person.
5. Sleep Duration (hours): The number of hours the person sleeps per day.
6. Quality of Sleep (scale: 1-10): A subjective rating of the quality of sleep, ranging from 1 to 10.
7. Physical Activity Level (minutes/day): The number of minutes the person engages in physical activity daily.
8. Stress Level (scale: 1-10): A subjective rating of the stress level experienced by the person, ranging from 1 to 10.
9. BMI Category: The BMI category of the person (e.g., Underweight, Normal, Overweight).
10. Blood Pressure (systolic/diastolic): The blood pressure measurement of the person, indicated as systolic pressure over diastolic pressure.
11. Heart Rate (bpm): The resting heart rate of the person in beats per minute.
12. Daily Steps: The number of steps the person takes per day.
13. Sleep Disorder: The presence or absence of a sleep disorder in the person (None, Insomnia, Sleep Apnea).

##### **Details about Sleep Disorder Column:**

1. None: The individual does not exhibit any specific sleep disorder.
2. Insomnia: The individual experiences difficulty falling asleep or staying asleep, leading to inadequate or poor-quality sleep.
3. Sleep Apnea: The individual suffers from pauses in breathing during sleep, resulting in disrupted sleep patterns and potential health risks.

##### **Checking the Columns Names**

In [63]:
df.columns

Index(['Person ID', 'Gender', 'Age', 'Occupation', 'Sleep Duration',
       'Quality of Sleep', 'Physical Activity Level', 'Stress Level',
       'BMI Category', 'Blood Pressure', 'Heart Rate', 'Daily Steps',
       'Sleep Disorder'],
      dtype='object')

##### **Checking the Total Number of Missing Values**

In [64]:
df.isnull().sum().to_frame().rename(columns={0:"Total No. of Missing Values"})

,Total No. of Missing Values
Person ID,0
Gender,0
Age,0
Occupation,0
Sleep Duration,0
Quality of Sleep,0
Physical Activity Level,0
Stress Level,0
BMI Category,0
Blood Pressure,0


**Note:**
The **Sleep Disorder** variable has incorrectly captured 219 missing values. In the raw data, these missing values are labeled as 'None'. To fix this, we replace "NaN" with "None".

In [65]:
df['Sleep Disorder'].fillna("None",inplace=True)

print(df['Sleep Disorder'].value_counts())

Sleep Disorder
None           219
Sleep Apnea     78
Insomnia        77
Name: count, dtype: int64


In [66]:
# Total No. of Missing Values after NaN replacement
df.isnull().sum().to_frame().rename(columns={0:"Total No. of Missing Values after NaN replacement"})

,Total No. of Missing Values after NaN replacement
Person ID,0
Gender,0
Age,0
Occupation,0
Sleep Duration,0
Quality of Sleep,0
Physical Activity Level,0
Stress Level,0
BMI Category,0
Blood Pressure,0


##### **Standardize BMI Category**

To standardize the "BMI Category" so that both "Normal" and "Normal Weight" are categorized as "Normal,".

In [67]:
df['BMI Category'] = df['BMI Category'].replace('Normal Weight','Normal')

##### **Splitting Blood Pressure to Two Columns**

I'm splitting the Blood Pressure to two columns as i dont see any significant if we have so many columns of different Blood Pressure readings.

In [68]:
if 'Blood Pressure' in df.columns:
    # Split the 'Blood Pressure' column into 'Systolic' and 'Diastolic' by splitting on the '/'
    df[['Systolic', 'Diastolic']] = df['Blood Pressure'].str.split('/', expand=True)

    # Convert the new columns to numeric type for analysis
    df['Systolic'] = pd.to_numeric(df['Systolic'], errors='coerce')
    df['Diastolic'] = pd.to_numeric(df['Diastolic'], errors='coerce')

    # Drop the original 'Blood Pressure' column now that we have numerical representations
    data = df.drop(columns=['Blood Pressure'])

data[['Systolic', 'Diastolic']].head()


,Systolic,Diastolic
0,126,83
1,125,80
2,125,80
3,140,90
4,140,90


In [69]:
# Drop unnecessary columns and encode categorical variables
data_new = data.drop(columns=['Person ID'])
data_new = pd.get_dummies(data_new, drop_first=True)

# Convert all boolean columns to integer (1 for True, 0 for False)
data_new = data_new.apply(lambda x: x.astype(int) if x.dtype == 'bool' else x)


data_new

,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps,Systolic,Diastolic,Gender_Male,...,Occupation_Nurse,Occupation_Sales Representative,Occupation_Salesperson,Occupation_Scientist,Occupation_Software Engineer,Occupation_Teacher,BMI Category_Obese,BMI Category_Overweight,Sleep Disorder_None,Sleep Disorder_Sleep Apnea
0,27,6.1,6,42,6,77,4200,126,83,1,...,0,0,0,0,1,0,0,1,1,0
1,28,6.2,6,60,8,75,10000,125,80,1,...,0,0,0,0,0,0,0,0,1,0
2,28,6.2,6,60,8,75,10000,125,80,1,...,0,0,0,0,0,0,0,0,1,0
3,28,5.9,4,30,8,85,3000,140,90,1,...,0,1,0,0,0,0,1,0,0,1
4,28,5.9,4,30,8,85,3000,140,90,1,...,0,1,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,59,8.1,9,75,3,68,7000,140,95,0,...,1,0,0,0,0,0,0,1,0,1
370,59,8.0,9,75,3,68,7000,140,95,0,...,1,0,0,0,0,0,0,1,0,1
371,59,8.1,9,75,3,68,7000,140,95,0,...,1,0,0,0,0,0,0,1,0,1
372,59,8.1,9,75,3,68,7000,140,95,0,...,1,0,0,0,0,0,0,1,0,1


##### **Target Variable**

The target variable (dependent variable) should be continuous, so I chose Sleep Duration over Quality of Sleep as the target. This is because Quality of Sleep is rated on a 1–10 discrete ordinal scale, which may be better suited for ordinal regression.

## **The Models**

### **Multiple Linear Regression Model**

As per the purpose of the learning, I have used different approaches to build an MLR model.

1. First, I will perform MLR using the statsmodels OLS model.
2. Second, I will perform MLR using sklearn's LinearRegression model.
3. Then, I will compare the results of the two approaches.

In [70]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm

In [71]:
X = data_new.drop(columns=['Quality of Sleep'])
y = data_new['Quality of Sleep']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### **1. statsmodels OLS Model**

In [72]:
X_ols = sm.add_constant(X)

# Fit the OLS model
ols_model = sm.OLS(y, X_ols).fit()

# Display the summary of the OLS model, including coefficients, p-values, and other statistics
ols_summary = ols_model.summary()
ols_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Quality of Sleep   R-squared:                       0.967
Model:                            OLS   Adj. R-squared:                  0.965
Method:                 Least Squares   F-statistic:                     445.2
Date:                Fri, 08 Nov 2024   Prob (F-statistic):          2.15e-243
Time:                        23:42:44   Log-Likelihood:                 40.183
No. Observations:                 374   AIC:                            -32.37
Df Residuals:                     350   BIC:                             61.82
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                               7.9485      1.022      7.777      0.000       5.938       9.959
Age                                 0.0596      0.005     11.664      0.000       0.050       0.070
Sleep Duration                      0.2441      0.047      5.144      0.000       0.151       0.337
Physical Activity Level            -0.0011      0.001     -0.750      0.453      -0.004       0.002
Stress Level                       -0.3723      0.026    -14.533      0.000      -0.423      -0.322
Heart Rate                         -0.0262      0.009     -2.815      0.005      -0.044      -0.008
Daily Steps                      3.352e-05   2.09e-05      1.606      0.109   -7.53e-06    7.46e-05
Systolic                           -0.0012      0.016     -0.074      0.941      -0.032       0.030
Diastolic                          -0.0103      0.021     -0.491      0.624      -0.052       0.031
Gender_Male                         0.5959      0.071      8.391      0.000       0.456       0.736
Occupation_Doctor                  -0.5520      0.084     -6.587      0.000      -0.717      -0.387
Occupation_Engineer                -0.6901      0.080     -8.577      0.000      -0.848      -0.532
Occupation_Lawyer                  -0.4173      0.095     -4.381      0.000      -0.605      -0.230
Occupation_Manager                 -0.3902      0.238     -1.637      0.103      -0.859       0.079
Occupation_Nurse                   -0.3057      0.100     -3.069      0.002      -0.502      -0.110
Occupation_Sales Representative    -1.5615      0.212     -7.373      0.000      -1.978      -1.145
Occupation_Salesperson             -0.8967      0.092     -9.696      0.000      -1.079      -0.715
Occupation_Scientist               -0.6231      0.154     -4.055      0.000      -0.925      -0.321
Occupation_Software Engineer       -0.5075      0.137     -3.709      0.000      -0.777      -0.238
Occupation_Teacher                 -0.5166      0.076     -6.816      0.000      -0.666      -0.368
BMI Category_Obese                 -0.0984      0.179     -0.549      0.583      -0.451       0.254
BMI Category_Overweight            -0.4556      0.088     -5.169      0.000      -0.629      -0.282
Sleep Disorder_None                 0.2271      0.054      4.235      0.000       0.122       0.333
Sleep Disorder_Sleep Apnea          0.2383      0.059      4.039      0.000       0.122       0.354
==============================================================================
Omnibus:                       78.359   Durbin-Watson:                   1.847
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              738.074
Skew:                          -0.550   Prob(JB):                    5.36e-161
Kurtosis:                       9.794   Cond. No.                  

In [60]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Display the VIF results
print(vif_data)

                            Feature           VIF
0                               Age    409.766338
1                  Quality of Sleep    812.363449
2           Physical Activity Level     52.580498
3                      Stress Level    235.244141
4                        Heart Rate   1389.039191
5                       Daily Steps    129.136996
6                          Systolic  23999.875924
7                         Diastolic  21324.594553
8                       Gender_Male     20.896321
9                 Occupation_Doctor      8.669197
10              Occupation_Engineer      7.899850
11                Occupation_Lawyer      7.086329
12               Occupation_Manager      1.124386
13                 Occupation_Nurse     12.355081
14  Occupation_Sales Representative      1.817115
15           Occupation_Salesperson      5.877207
16             Occupation_Scientist      1.830022
17     Occupation_Software Engineer      1.464473
18               Occupation_Teacher      4.986351


In [76]:
features_to_remove = ['Systolic', 'Diastolic']

# Drop the features from the DataFrame
df_reduced1 = data_new.drop(columns=features_to_remove)

# Display the first few rows of the updated DataFrame to check the result

   Age  Sleep Duration  Quality of Sleep  Physical Activity Level  \
0   27             6.1                 6                       42   
1   28             6.2                 6                       60   
2   28             6.2                 6                       60   
3   28             5.9                 4                       30   
4   28             5.9                 4                       30   

   Stress Level  Heart Rate  Daily Steps  Gender_Male  Occupation_Doctor  \
0             6          77         4200            1                  0   
1             8          75        10000            1                  1   
2             8          75        10000            1                  1   
3             8          85         3000            1                  0   
4             8          85         3000            1                  0   

   Occupation_Engineer  ...  Occupation_Nurse  \
0                    0  ...                 0   
1                    0  ...   

In [77]:
# Define the feature set (X) without the dropped features
X = df_reduced1.drop(columns=['Quality of Sleep'])  # Replace with your target variable name

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Display the VIF results
print(vif_data)

                            Feature         VIF
0                               Age  173.405880
1                    Sleep Duration  636.458503
2           Physical Activity Level   48.732304
3                      Stress Level  143.776474
4                        Heart Rate  984.144616
5                       Daily Steps   92.462183
6                       Gender_Male   15.796618
7                 Occupation_Doctor    9.089239
8               Occupation_Engineer    6.591525
9                 Occupation_Lawyer    5.859333
10               Occupation_Manager    1.101640
11                 Occupation_Nurse    6.967365
12  Occupation_Sales Representative    1.637384
13           Occupation_Salesperson    5.222435
14             Occupation_Scientist    1.585057
15     Occupation_Software Engineer    1.407211
16               Occupation_Teacher    3.178602
17               BMI Category_Obese    3.972546
18          BMI Category_Overweight   14.112009
19              Sleep Disorder_None   10

In [81]:
features_to_remove = ['Physical Activity Level']

# Drop the features from the DataFrame
df_reduced2 = df_reduced1.drop(columns=features_to_remove)


In [82]:
# Define the feature set (X) without the dropped features
X = df_reduced2.drop(columns=['Quality of Sleep'])  # Replace with your target variable name

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Display the VIF results
print(vif_data)

                            Feature         VIF
0                               Age  171.012210
1                    Sleep Duration  621.712397
2                      Stress Level  142.010192
3                        Heart Rate  979.913640
4                       Daily Steps   36.613900
5                       Gender_Male   13.698638
6                 Occupation_Doctor    8.354350
7               Occupation_Engineer    6.384051
8                 Occupation_Lawyer    5.545781
9                Occupation_Manager    1.097977
10                 Occupation_Nurse    6.770072
11  Occupation_Sales Representative    1.541674
12           Occupation_Salesperson    4.812944
13             Occupation_Scientist    1.584512
14     Occupation_Software Engineer    1.322088
15               Occupation_Teacher    3.028556
16               BMI Category_Obese    3.442055
17          BMI Category_Overweight   13.940271
18              Sleep Disorder_None   10.595755
19       Sleep Disorder_Sleep Apnea    5

In [83]:
features_to_remove = ['Daily Steps']

# Drop the features from the DataFrame
df_reduced3 = df_reduced2.drop(columns=features_to_remove)

In [84]:
# Define the feature set (X) without the dropped features
X = df_reduced3.drop(columns=['Quality of Sleep'])  # Replace with your target variable name

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Display the VIF results
print(vif_data)

                            Feature         VIF
0                               Age  169.670482
1                    Sleep Duration  578.085035
2                      Stress Level  127.992603
3                        Heart Rate  953.020431
4                       Gender_Male   13.246154
5                 Occupation_Doctor    7.698021
6               Occupation_Engineer    5.754334
7                 Occupation_Lawyer    5.476248
8                Occupation_Manager    1.096658
9                  Occupation_Nurse    6.732358
10  Occupation_Sales Representative    1.511614
11           Occupation_Salesperson    4.522573
12             Occupation_Scientist    1.576984
13     Occupation_Software Engineer    1.306834
14               Occupation_Teacher    3.027037
15               BMI Category_Obese    3.354183
16          BMI Category_Overweight   13.892899
17              Sleep Disorder_None   10.466862
18       Sleep Disorder_Sleep Apnea    4.945389


### **2. sklearn's LinearRegression Model**

In [31]:
# Initialize and fit the regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mse, r2

(0.07451552407748639, 0.8880762884154006)

**Insights:**

An R² value of 0.888 indicates that the model explains about 88.83% of the variance in "Sleep Duration," suggesting a strong fit. The low MSE also indicates that the model's predictions are close to the actual values on average.

**Two Approaches Comparison**

1. sklearn's LinearRegression Model: When you used sklearn's LinearRegression, it split the data into training and testing sets. The R2 score calculated in that case (0.888) was only for the testing set, reflecting the model's ability to generalize to unseen data. This gives a more realistic assessment of the model's performance.

2. statsmodels OLS Model: The OLS approach with statsmodels used the entire dataset for training and evaluation, resulting in a slightly higher R2  value (0.912). Since it's calculated on the full dataset, the model fits the data better but may not generalize as well to new data.

### **Variable Selection**

In [17]:
import statsmodels.api as sm

# Define a function to perform forward selection
def forward_selection(X, y, significance_level=0.05):
    initial_features = []
    remaining_features = list(X.columns)
    best_features = []
    while remaining_features:
        best_pval = float("inf")
        best_feature = None
        for feature in remaining_features:
            model = sm.OLS(y, sm.add_constant(X[initial_features + [feature]])).fit()
            pval = model.pvalues[feature]
            if pval < best_pval:
                best_pval = pval
                best_feature = feature
        if best_pval < significance_level:
            initial_features.append(best_feature)
            remaining_features.remove(best_feature)
            best_features.append(best_feature)
        else:
            break
    return best_features

# Define a function to perform backward elimination
def backward_elimination(X, y, significance_level=0.05):
    features = list(X.columns)
    while features:
        model = sm.OLS(y, sm.add_constant(X[features])).fit()
        pvalues = model.pvalues.iloc[1:]  # Exclude the intercept
        max_pval = pvalues.max()
        if max_pval > significance_level:
            excluded_feature = pvalues.idxmax()
            features.remove(excluded_feature)
        else:
            break
    return features

# Define a function to perform stepwise selection (combining forward and backward)
def stepwise_selection(X, y, significance_level=0.05):
    initial_features = []
    remaining_features = list(X.columns)
    best_features = []
    while remaining_features or initial_features:
        changed = False

        # Forward step
        forward_best_pval = float("inf")
        best_feature = None
        for feature in remaining_features:
            model = sm.OLS(y, sm.add_constant(X[initial_features + [feature]])).fit()
            pval = model.pvalues[feature]
            if pval < forward_best_pval:
                forward_best_pval = pval
                best_feature = feature
        if forward_best_pval < significance_level:
            initial_features.append(best_feature)
            remaining_features.remove(best_feature)
            best_features.append(best_feature)
            changed = True

        # Backward step
        model = sm.OLS(y, sm.add_constant(X[initial_features])).fit()
        pvalues = model.pvalues.iloc[1:]  # Exclude the intercept
        max_pval = pvalues.max()
        if max_pval > significance_level:
            excluded_feature = pvalues.idxmax()
            initial_features.remove(excluded_feature)
            remaining_features.append(excluded_feature)
            best_features.remove(excluded_feature)
            changed = True

        if not changed:
            break
    return best_features

# Run variable selection methods
forward_selected_features = forward_selection(X_train, y_train)
backward_selected_features = backward_elimination(X_train, y_train)
stepwise_selected_features = stepwise_selection(X_train, y_train)

# Define a function to evaluate a model based on selected features
def evaluate_model(selected_features, X_train, X_test, y_train, y_test):
    # Fit model with selected features
    model = LinearRegression()
    model.fit(X_train[selected_features], y_train)

    # Make predictions and calculate performance metrics
    y_pred = model.predict(X_test[selected_features])
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Calculate adjusted R^2
    n = X_test[selected_features].shape[0]  # Number of observations
    p = len(selected_features)  # Number of predictors
    adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))

    return mse, r2, adjusted_r2

# Evaluate models for forward, backward, and stepwise selected features
forward_mse, forward_r2, forward_adjusted_r2 = evaluate_model(forward_selected_features, X_train, X_test, y_train, y_test)
backward_mse, backward_r2, backward_adjusted_r2 = evaluate_model(backward_selected_features, X_train, X_test, y_train, y_test)
stepwise_mse, stepwise_r2, stepwise_adjusted_r2 = evaluate_model(stepwise_selected_features, X_train, X_test, y_train, y_test)

# Print results
print("Forward Selected Features:\n" +
      "MSE: " + str(forward_mse) + "\nR2: " + str(forward_r2) + "\nAdjusted R2: " + str(forward_adjusted_r2) + "\n" + "\n".join(forward_selected_features) + "\n")
print("Backward Selected Features:\n" +
      "MSE: " + str(backward_mse) + "\nR2: " + str(backward_r2) + "\nAdjusted R2: " + str(backward_adjusted_r2) + "\n" + "\n".join(backward_selected_features) + "\n")
print("Stepwise Selected Features:\n" +
      "MSE: " + str(stepwise_mse) + "\nR2: " + str(stepwise_r2) + "\nAdjusted R2: " + str(stepwise_adjusted_r2) + "\n" + "\n".join(stepwise_selected_features))

Forward Selected Features:
MSE: 0.049241478987893626
R2: 0.9673599670861535
Adjusted R2: 0.9590616536334806
Stress Level
Sleep Duration
Occupation_Teacher
Daily Steps
Occupation_Scientist
Occupation_Salesperson
Occupation_Sales Representative
Age
BMI Category_Overweight
Occupation_Lawyer
Diastolic
Gender_Male
Occupation_Engineer
Occupation_Doctor
Heart Rate

Backward Selected Features:
MSE: 0.05978383258550727
R2: 0.9603718998004386
Adjusted R2: 0.9494400100902147
Age
Sleep Duration
Stress Level
Heart Rate
Gender_Male
Occupation_Doctor
Occupation_Engineer
Occupation_Lawyer
Occupation_Nurse
Occupation_Sales Representative
Occupation_Salesperson
Occupation_Scientist
Occupation_Teacher
BMI Category_Overweight
Sleep Disorder_None
Sleep Disorder_Sleep Apnea

Stepwise Selected Features:
MSE: 0.04924147898789539
R2: 0.9673599670861524
Adjusted R2: 0.9590616536334792
Stress Level
Sleep Duration
Occupation_Teacher
Daily Steps
Occupation_Scientist
Occupation_Salesperson
Occupation_Sales Represen

### **The Best MLR Model**

Backward Elimination appears to be the best model in this case:

- It has the lowest MSE (0.0725), suggesting it makes the most accurate predictions.
- It has the highest R2 (0.8911), meaning it explains the most variance in the target variable.
- It has the highest adjusted R2 (0.8587), indicating a good balance between predictive power and the number of predictors.

Therefore, the **Backward Elimination model is the best MLR model.**

### **Model Diagnostics (Residual Plots)**